In [ ]:
! pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyRegressor

## **Cargar dataset**


In [ ]:
df_leishmaniasis = pd.read_csv('leishmaniasis_06_bioactivity_data_3class_pIC50_pubchem_fp.csv')
df1 = pd.read_csv('leishmaniasis_07_bioactivity_data_3class_pIC50_morgan_fp.csv')

df_leishmaniasis.head(10)

FileNotFoundError: ignored

In [ ]:
df_leishmaniasis.shape

(7049, 882)

In [ ]:
#Definir la matriz con variables explicativas y el vector respuesta
Xl = df_leishmaniasis.drop('pIC50', axis=1)
Yl = df_leishmaniasis.pIC50

In [ ]:
Yl.describe()

count   7049.00
mean       5.60
std        1.67
min        1.76
25%        4.30
50%        4.82
75%        7.00
max       10.24
Name: pIC50, dtype: float64

##**Definir el conjunto de entrenamiento y de test**

In [ ]:
#Definir os conjuntos de entrenamiento y test
Xl_train, Xl_test, Yl_train, Yl_test = train_test_split(Xl, Yl, test_size=0.2, random_state=42)

In [ ]:
print(Yl_train)

315    4.49
4561   4.72
5007   6.50
1161   4.84
2275   6.75
       ... 
3772   8.24
5191   6.30
5226   7.00
5390   4.60
860    7.82
Name: pIC50, Length: 5639, dtype: float64


##**Establecer el modelo con lazyregresor**

In [ ]:
#Se establece el modelo
modelo = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models_train, predictions_train = modelo.fit(Xl_train, Xl_train, Yl_train, Yl_train)
models_test, predictions_test = modelo.fit(Xl_train, Xl_test, Yl_train, Yl_test) #35 mins

 74%|███████▍  | 31/42 [12:09<01:00,  5.53s/it]

In [ ]:
predictions_train

In [ ]:
#Importar las librerías para ajustar el modelo
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [ ]:
Xl_train.shape[1]

In [ ]:
#Ajuste del modelo 
model2=DecisionTreeRegressor(random_state=120).fit(Xl_train,Yl_train)
model2

In [ ]:
#score o R@2
model2.score(Xl_train,Yl_train)

##**Predicción modelo**

In [ ]:
model2.predict(Xl_test)

##**PCA sobre Xl**

In [ ]:
# PCA sobre Xl
from sklearn.decomposition import PCA
# Loop Function to identify number of principal components that explain at least 95% of the variance
for comp in range(3, Xl.shape[1]):
    pca = PCA(n_components= comp, random_state=159)
    pca.fit(Xl)
    comp_check = pca.explained_variance_ratio_
    final_comp = comp
    if comp_check.sum() > 0.95:
        break
        
Final_PCA_Xl = PCA(n_components= final_comp,random_state=42)
Final_PCA_Xl.fit(Xl)
PCA_Xl=Final_PCA_Xl.transform(Xl)
num_comps = comp_check.shape[0]

In [ ]:
print("Usando {} componentes, se puede explicar el {}% de la data original.".format(final_comp,comp_check.sum()))

##**Se usan 3 compontentes para explicar el 99% de la varianza**

In [ ]:
PCA_Xl

##**Visualización de métricas**

In [ ]:
# Bar plot of R-squared values
import matplotlib.pyplot as plt
import seaborn as sns

#train["R-Squared"] = [0 if i < 0 else i for i in train.iloc[:,0] ]

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=predictions_train.index, x="R-Squared", data=predictions_train)
ax.set(xlim=(0, 1))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=predictions_train.index, x="Time Taken", data=predictions_train)
ax.set(xlim=(0, 10))

##**Modelo de clasificación(Variables reducidas)**

In [ ]:
#Modelo de clasificación 
import sklearn.linear_model
from sklearn.linear_model import LogisticRegression
import numpy as np
import random as rm

In [ ]:
Xp1_train, Xp1_test, Yp1_train, Yp1_test = train_test_split(PCA_Xl, Yl, test_size=0.2, random_state=42)

In [ ]:
#Descripción de los datos de entrenamiento
Yp1_train.describe()

In [ ]:
#Definir el treshold para clasificar el valor de IPC50 en 1 o 0
Yp2_train= [1 if i >= Yl.mean() else 0 for i in Yp1_train]

In [ ]:
#Ajuste del modelo de regresión logística
#Modelo de clasificación
Model_clasification_pca=LogisticRegression(random_state=0).fit(Xp1_train, Yp2_train)

In [ ]:
#Información del modelo
print("Intercepto:", Model_clasification_pca.intercept_)
print("Accuracy de entrenamiento:", Model_clasification_pca.score(Xp1_train, Yp2_train))

In [ ]:
import numpy as np
np.exp(-0.33037203) 

In [ ]:
np.exp(Model_clasification_pca.intercept_)

In [ ]:
# Predicciones probabilísticas
# ==============================================================================
# Con .predict_proba() se obtiene, para cada observación, la probabilidad predicha
# de pertenecer a cada una de las dos clases.
predicciones = Model_clasification_pca.predict_proba(X = Xp1_test)
predicciones = pd.DataFrame(predicciones, columns = Model_clasification_pca.classes_)
predicciones.head(20)

In [ ]:
# Predicciones con clasificación final
# ==============================================================================
# Con .predict() se obtiene, para cada observación, la clasificación predicha por
# el modelo. Esta clasificación se corresponde con la clase con mayor probabilidad.
Ypca_pred= Model_clasification_pca.predict(X = Xp1_test)
Ypca_pred

##**Matriz de confusiónd de datos reducidos**

In [ ]:
#Definir el treshold para clasificar el valor de IPC50 en 1 o 0
Ypca2_test= [1 if i >= Yl.mean() else 0 for i in Yp1_test]
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Ypca2_test, Ypca_pred)
Conf_matrix=print(confusion_matrix)

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(Ypca2_test, Ypca_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1],)

cm_display.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Ypca2_test, Ypca_pred))

In [ ]:
Yp2_train

In [ ]:
import matplotlib.pyplot as plt


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
clf = SVC(random_state=0)
clf.fit(Xp1_train, Yp2_train)

SVC(random_state=0)
predictions_pca = clf.predict(Xp1_test)

predictions_pca

In [ ]:
#Matriz de confusión del SVM 
Conf_matrix= confusion_matrix(Ypca2_test, predictions_pca, labels=clf.classes_)
Conf_matrix

In [ ]:
#Visualización de la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=Conf_matrix,display_labels=clf.classes_)
disp.plot()
plt.show()

##**Ajuste del modelo (Variables sin reducción de dimensión)**

In [ ]:
Yl1_train= [1 if i >= Yl.mean() else 0 for i in Yl_train]

In [ ]:
#Definir el treshold para clasificar el valor de IPC50 en 1 o 0
Yp2_train= [1 if i >= Yl.mean() else 0 for i in Yp1_train]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#Modelo de clasificación
Model_clasification_leishmaniasis=LogisticRegression(random_state=0).fit(Xl_train, Yl1_train)

In [ ]:
#Información del modelo
print("Intercepto:", Model_clasification_leishmaniasis.intercept_)

print("Accuracy de entrenamiento:", Model_clasification_leishmaniasis.score(Xl_train, Yl1_train))

In [ ]:
np.exp(Model_clasification_leishmaniasis.intercept_)

In [ ]:
# Predicciones probabilísticas
# ==============================================================================
# Con .predict_proba() se obtiene, para cada observación, la probabilidad predicha
# de pertenecer a cada una de las dos clases.
predicciones = Model_clasification_leishmaniasis.predict_proba(X = Xl_test)
predicciones = pd.DataFrame(predicciones, columns = Model_clasification_leishmaniasis.classes_)
predicciones.head(20)

In [ ]:
# Predicciones con clasificación final
# ==============================================================================
# Con .predict() se obtiene, para cada observación, la clasificación predicha por
# el modelo. Esta clasificación se corresponde con la clase con mayor probabilidad.
Y_pred_c= Model_clasification_leishmaniasis.predict(X = Xl_test)
Y_pred_c

##**Matriz de confusión**

In [ ]:
Yl1_test=[1 if i >= Yl.mean() else 0 for i in Yl_test]

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Yl1_test, Y_pred_c)
Conf_matrix=print(confusion_matrix)

##**Matriz de forma gráfica**

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(Yl1_test, Y_pred_c)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1],)

cm_display.plot()
plt.show()

##**Métricas**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Yl1_test, Y_pred_c))

## **Modelo de clasificación usando SVM(SUPPORT VECTOR MACHINES)**

In [ ]:
import matplotlib.pyplot as plt


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
clf = SVC(random_state=0)
clf.fit(Xl_train, Yl1_train)

SVC(random_state=0)
predictions_leishmaniasis = clf.predict(Xl_test)

predictions_leishmaniasis

In [ ]:
#Matriz de confusión del SVM 
cm = confusion_matrix(Yl1_test, predictions_leishmaniasis, labels=clf.classes_)
cm

In [ ]:
#Visualización de la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=clf.classes_)
disp.plot()
plt.show()